In [4]:
# data analysis and wrangling
import numpy as np
import pandas as pd
import json

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold




from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

# ignore Warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_df = pd.read_csv("preprocessed_train.csv", encoding='euc-kr')

In [ ]:
train_df.head()

In [ ]:
train_df = train_df.drop(columns=['분석데이터'], axis=1)

In [ ]:
x_train = train_df[:8000]
y_train = x_train['label']

x_test = train_df[8000:]
y_test = x_test['label']


x_train = x_train.drop(columns=['label'], axis=1)
x_test = x_test.drop(columns=['label'], axis=1)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

## 데이터 정형화

In [ ]:
maxCol = x_train.max()
col = x_train.columns
for i in range(len(maxCol)):
    x_train[col[i]] = x_train[col[i]] / maxCol[i]

In [ ]:
x_train

In [ ]:
x = x_train.to_numpy()

x1 = np.zeros((8000,22,28))
x2 = np.zeros((8000,22,28,1))

for i in range(len(x_train)):
    x1[i] = x[i].reshape((22,28))
    for j in range(22):
        for k in range(28):
            x2[i][j][k][0] = x1[i][j][k]
            
x_train = x2

In [ ]:
x1.shape

In [ ]:
y_train = y_train.to_numpy()

In [ ]:
maxCol2 = x_test.max()
col2 = x_test.columns
for i in range(len(maxCol2)):
    x_test[col[i]] = x_test[col[i]] / maxCol2[i] 

In [ ]:
x_test

In [ ]:
xt = x_test.to_numpy()

xt1 = np.zeros((2000,22,28))
xt2 = np.zeros((2000,22,28,1))

for i in range(len(x_test)):
    xt1[i] = xt[i].reshape((22,28))
    for j in range(22):
        for k in range(28):
            xt2[i][j][k][0] = xt1[i][j][k]
            
x_test = xt2

In [ ]:
y_test = y_test.to_numpy()

In [ ]:
x_test.shape

In [ ]:
# 합성곱 층 만들기

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(22, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [ ]:
# 마지막에 Dense 층 추가하기
#model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.25))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# 모델 컴파일과 훈련하기
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

In [ ]:
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(test_acc)

In [ ]:
from matplotlib import pyplot as plt

for i in range(50):
    plt.imshow(x_test[i], cmap='gray', interpolation='nearest')
    plt.title(str(y_test[i]))
    plt.show()